<a href="https://colab.research.google.com/github/yeesem/Advanced-Tensorflow-Specialization/blob/main/OxfordPets_UNet_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt
import numpy as np

# Download the Oxford-IIIT Pets Dataset

In [2]:
# If you hit a problem with checksums, you can execute the following line first
!python -m tensorflow_datasets.scripts.download_and_prepare --register_checksums --datasets=oxford_iiit_pet:3.1.0

# download the dataset and get info
dataset, info = tfds.load('oxford_iiit_pet:3.*.*', with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/3680 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_iiit_pet/incomplete.O4YUQI_3.2.0/oxford_iiit_pet-train.tfrecord*...…

Generating test examples...:   0%|          | 0/3669 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_iiit_pet/incomplete.O4YUQI_3.2.0/oxford_iiit_pet-test.tfrecord*...:…

Dataset oxford_iiit_pet downloaded and prepared to /root/tensorflow_datasets/oxford_iiit_pet/3.2.0. Subsequent calls will reuse this data.


In [3]:
# Possible keys can access in the dataset dict
# contains test and train splits
print(dataset.keys())

dict_keys(['train', 'test'])


In [4]:
# See information about the dataset
print(info)

tfds.core.DatasetInfo(
    name='oxford_iiit_pet',
    full_name='oxford_iiit_pet/3.2.0',
    description="""
    The Oxford-IIIT pet dataset is a 37 category pet image dataset with roughly 200
    images for each class. The images have large variations in scale, pose and
    lighting. All images have an associated ground truth annotation of breed.
    """,
    homepage='http://www.robots.ox.ac.uk/~vgg/data/pets/',
    data_dir=PosixGPath('/tmp/tmpw2y4q601tfds'),
    file_format=tfrecord,
    download_size=773.52 MiB,
    dataset_size=774.69 MiB,
    features=FeaturesDict({
        'file_name': Text(shape=(), dtype=string),
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=37),
        'segmentation_mask': Image(shape=(None, None, 1), dtype=uint8),
        'species': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        '

# Prepare the Dataset

In [5]:
# Preprocessing Utilities

# Data Augmentation
def random_flip(input_image,input_mask):
  if tf.random.uniform(()) > 0.5:
    input_image = tf.image.flip_left_right(input_image)
    input_mask = tf.image.flip_left_right(input_image)

  return input_image,input_mask

# Data Normalization
def normalize(input_image,input_mask):
  input_image = tf.cast(input_image,tf.float32) / 255.0
  input_mask -= 1
  return input_image,input_mask

@tf.function
def load_image_train(datapoint):
  input_image = tf.image.resize(datapoint['image'],(128,128),method = 'nearest')
  input_mask = tf.image.resize(datapoint['segmentation_mask'],(128,128),method = 'nearest')
  input_image,input_mask = random_flip(input_image,input_mask)
  input_image,input_mask = normalize(input_image,input_mask)

  return input_image,input_mask

def load_image_test(datapoint):
  input_image = tf.image.resize(datapoint['image'],(128,128),method = 'nearest')
  input_mask = tf.image.resize(datapoint['image'],(128,128),method = 'nearest')
  input_image,input_mask = normalize(input_image,input_mask)

  return input_image,input_mask

In [7]:
# Preprocess the train and test sets
# num_parallel_calls: This parameter controls the level of parallelism. It determines how many
# elements of the dataset are processed concurrently.
# tf.data.experimental.AUTOTUNE is a special setting that allows TensorFlow to automatically decide the
# optimal number of parallel calls. It uses available CPU resources to dynamically determine the best
# parallelism level for performance, potentially speeding up data preprocessing.
train = dataset['train'].map(load_image_train,num_parallel_calls = tf.data.experimental.AUTOTUNE)
test = dataset['test'].map(load_image_test)

In [8]:
BATCH_SIZE = 64
BUFFER_SIZE = 1000

# Shuffle and group the train set into batches
train_dataset = train.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
# Do a prefetch to optimzie preprocessing
train_dataset = train_dataset.prefetch(buffer_size = tf.data.experimental.AUTOTUNE)

# group the test set into batches
test_dataset = test.batch(BATCH_SIZE)